# map_partitions

## Learning Objectives

In this tutorial, you will learn:

  * how to apply a user-defined function using distributed computing across an entire LSDB catalog's partitions
  * when, why, and how to supply required Dask "meta" when it cannot be inferred from your user-defined function

## Introduction

LSDB/HATS catalogs are organized into partitions, and this number of partitions is reported as `npartitions=` in the header, whenever printing a catalog.  Each partition corresponds to a single pixel, and they have been sized to have approximately the same number of rows in each partition, in order to enable efficient parallel computation.

The `map_partitions` method provides a means for users to execute their own analysis functions on each *partition* of the catalog data. The data will be passed to your function as a Pandas DataFrame (`pd.DataFrame`) object.

## 1. Open a catalog

For this example, we will use a small box of Data Preview 1, and only specify columns of interest. This limits the overall memory requirements of the pipeline.

In [ ]:
# Dask puts out more advisory logging than we care for in this tutorial.
# It takes some doing to quiet all of it, but this recipe works.

import dask

dask.config.set({"logging.distributed": "critical"})

import logging

# This also has to be done, for the above to be effective
logger = logging.getLogger("distributed")
logger.setLevel(logging.CRITICAL)

import warnings

# Finally, suppress the specific warning about Dask dashboard port usage
warnings.filterwarnings("ignore", message="Port 8787 is already in use.")

In [ ]:
import lsdb

dp1 = lsdb.open_catalog("https://data.lsdb.io/hats/mock_dp1/object_collection",
                        search_filter=lsdb.BoxSearch(ra=[51.,53.], dec=[-30.,-28.5]),
                       columns=["objectId", "coord_ra", "coord_dec", "g_psfMag", "i_psfMag"])
dp1

In [ ]:
# You can get the number of partitions programmatically this way.
# This can be valuable when you want to choose the optimal number
# of workers to process the partitions.
dp1.npartitions

In [ ]:
# You can also access the individual HealpixPixel objects for
# each partition, this way, inspecting their order and pixel,
# if desired.
px = dp1.get_healpix_pixels()[0]
px.order, px.pixel

## 2. Generating New Columns

Since the partition's `pd.DataFrame` is passed in to your custom function, you can augment it with new columns based on the existing columns, in ordinary Pandas style.

### 2.1 What you can map

The trick is understanding what kind of custom function you can pass to `.map_partitions`.  Your function is going to receive a Pandas DataFrame as its first parameter.  Other parameters can be passed in as keyword arguments to `.map_partitions`, as you'll see later on.  For now, we'll use a function that takes in one partition and produces a result that has the same shape.

Because the catalog is loaded lazily, `.map_partitions` also returns a lazy, or unevaluated, result.  You can see the results the same way you can realize the original catalog, by any of these means:
  * calling `.compute()` to produce a `pd.DataFrame` in memory;
  * calling `.to_hats()` to serialize it to disk as a HATS-format file;
  * calling `.head()` to see the first few rows, `.tail()` to see the last few.

In [ ]:
def g_minus_i(df, pixel):
    df["psf_g_minus_i"] = df["g_psfMag"] - df["i_psfMag"]
    return df


unrealized = dp1.map_partitions(g_minus_i, include_pixel=True)
unrealized

Taking a quick peek to see whether our function works correctly, and if the results in our new column are about what we expect:

In [ ]:
head_5 = unrealized.head(5)
head_5

Looks good! Now on to computing the complete result.

This unrealized result has a top-level property indicating how many partitions it has.  We can use this to choose our number of workers directly.

However, it's a good idea to bound the number of workers, in case the number of partitions is larger than we expect (or we move this code fragment elsewhere).

In [ ]:
%%time
from dask.distributed import Client

npartitions = dp1.npartitions

# Create a client which will be implicitly used until we make a new one
client = Client(n_workers=min(2, npartitions), memory_limit="auto")

result = unrealized.compute()
result

No reduction step is needed here since the operation is not a reducing operation.
There are as many rows in the new output as there were in the input.

## 3. Functions that reduce

The above works when your output rows are the same as your input rows.  When you're doing a reducing operation (such as calculating statistics), the process changes a little.

### 3.1. Your function's parameters

Again, your first input parameter is a `pd.DataFrame` that is one partition of the catalog, and the return value of your function needs to be the same, even if your result has only a single row.

If you want to know the HEALPix number of the partition, calling `.map_partitions` with `include_pixel=True` will pass that as the second parameter to your function.  We'll do this in this example, for demonstration purposes, though it isn't strictly necessary to this task.

If you have any other parameters that your function requires, take them as keyword arguments, and you can pass their values in as such, when calling `.map_partitions`.  Our example will do this, too, taking a `target_column=` argument.

### 3.2. What you get back

The operation we're going to do here is a reducing operation (min and max), and it will be run on each partition, reducing the many rows in each partition to a single value.  This means that the output of `.map_partitions` in this case will contain *one row per partition*.  Thus, you will need to do additional reduction on this output in order to get a single final result.

In [ ]:
# Note that this function must work correctly when given an empty DataFrame
# as an input, too; if not, you're obliged to provide "meta", that is,
# information about output type and shape.
import pandas as pd


def find_stats(df, pixel, target_column=""):
    c = df[target_column]
    min_val = c.min()
    max_val = c.max()
    mean_val = c.mean()
    return pd.DataFrame(
        [
            {
                "pixel": pixel,
                f"{target_column}_min": min_val,
                f"{target_column}_max": max_val,
                f"{target_column}_mean": mean_val,
            }
        ]
    )

### 3.3 When You Need `meta=`

The above definition of `find_stats` works even with an empty `pd.DataFrame` argument because `.mean()` is written to handle zero-row inputs without errors.

But your own custom function might not. As a trivial example, suppose you implemented the arithmetic mean yourself, as below.

In [ ]:
# This version of the function will NOT work with map_partitions as is, because
# of the attempt to divide by `c.count()`, which will be zero for an empty input.
import pandas as pd


def find_stats_needs_meta(df, pixel, target_column=""):
    c = df[target_column]
    min_val = c.min()
    max_val = c.max()
    # WARNING! c.count() == 0 when passed an empty DataFrame.
    # But meta= will come to the rescue.
    mean_val = c.sum() / c.count()
    return pd.DataFrame(
        [
            {
                "pixel": pixel,
                f"{target_column}_min": min_val,
                f"{target_column}_max": max_val,
                f"{target_column}_mean": mean_val,
            }
        ]
    )

In the above case, then, you need to indicate to Dask what type the output will be.

What Dask needs to know are the column names and their order, and so the below definition works, even though the types of the columns aren't indicated. (The type of `"pixel"` will default to `float64`, which is wrong, but doesn't matter in this case.)

In [ ]:
output_meta = pd.DataFrame(
    {
        "pixel": [],
        "g_psfMag_min": [],
        "g_psfMag_max": [],
        "g_psfMag_mean": [],
    }
)

Here's another definition of `output_meta` that works equally well, and has the advantage that the way the `pd.DataFrame` is initialized is the same form as the return value in the custom function. The type of `"pixel"` will now be more correct:

In [ ]:
output_meta = pd.DataFrame(
    [
        {
            "pixel": lsdb.HealpixPixel(0, 0),
            "g_psfMag_min": 0.0,
            "g_psfMag_max": 0.0,
            "g_psfMag_mean": 0.0,
        }
    ]
)

In [ ]:
output_meta["pixel"].dtype

although `pd.DataFrame` only cares that it is an "Object", as we can see.

These can be complicated and error-prone to construct, and small mistakes can create confusing errors that show up late in the computation.

To help with these difficulties, Dask does provide a `make_meta` function. If you can pass it a single valid row from your catalog data (that's what `head_5.head(1)` will do) which will work with your custom function, `make_meta` will generate the meta for it.

**NOTE:** It's much faster here to use the computed data from last time (`head_5`) than trying to use `dp1.head(1)`. The latter will still give the right answer, but will be much slower, since it obliges Dask to re-execute the computation.

In [ ]:
%%time
from dask.dataframe.utils import make_meta

output_meta = make_meta(
    find_stats_needs_meta(head_5.head(1), dp1.get_healpix_pixels()[0], target_column="g_psfMag")
)

Passing a correct `meta=` to `.map_partitions` will allow Dask to skip the process of sending your function an empty `pd.DataFrame`, and so, in our case of `find_stats_needs_meta` (where we depend on a non-zero `c.count()`), it will succeed without error.

In [ ]:
unrealized = dp1.map_partitions(
    find_stats_needs_meta,
    include_pixel=True,
    # Keyword arguments after 'include_pixel=' are passed to your function.
    target_column="g_psfMag",
    # Here we give Dask the hint it needs to avoid giving us an empty frame
    meta=output_meta,
)

In [ ]:
%%time
result = unrealized.compute()
result

The objects in the 'pixel' column are the same type as from `get_healpix_pixels()`.

In [ ]:
type(result["pixel"].iloc[0])

Because the result is one row per partition, we need additional reduction to get our single answer.

In [ ]:
result["g_psfMag_min"].min(), result["g_psfMag_max"].max()

What about searching not only the four partitions from our cone search, but the whole catalog?  All that changes is the number of partitions.

**NOTE** that since we are now using the `find_stats` that doesn't need `meta=`, we don't need to provide it.

In [ ]:
dp1_all = lsdb.open_catalog("https://data.lsdb.io/hats/mock_dp1/object_collection",
                           columns=["objectId", "coord_ra", "coord_dec", "g_psfMag", "i_psfMag"])
unrealized = dp1_all.map_partitions(
    find_stats,
    include_pixel=True,
    # Keyword arguments after 'include_pixel=' are passed to your
    # function
    target_column="g_psfMag",
)
npartitions = unrealized.npartitions

In [ ]:
npartitions

While not too many for Data Preview 1, for full Rubin and other large surveys this can be a huge number of partitions! If we didn't bound this value, we could easily overwhelm our cluster.

In [ ]:
%%time
# Close the old client and make a new one that takes into account the new number of partitions.
client.close()
client = Client(n_workers=1, memory_limit="auto") # 1 for RSP
result = unrealized.compute()
result

In [ ]:
# We need to do a final reduction step to get the true min and max
result["g_psfMag_min"].min(), result["g_psfMag_max"].max()

Since we just searched the whole catalog, we can check our answer
against the statistics that were compiled at import time for the
catalog.  As you can see, they match what we got when using
the `.map_partitions` method.

In [ ]:
dp1_all.hc_structure.aggregate_column_statistics(include_columns="g_psfMag")

## Closing the Dask client

In [ ]:
client.close()

## About

**Authors**: Derek Jones

**Last updated on**: April 17, 2025

If you use `lsdb` for published research, please cite following [instructions](https://docs.lsdb.io/en/stable/citation.html).